In [1]:
import pandas as pd
import numpy as np

import json
import requests
from IPython.display import clear_output

import psycopg2 as pg
conn = pg.connect(database="spotify_db",
                  user="postgres", 
                  password="damara1004")

def run_query(q):
    with conn:
        try:
            cur = conn.cursor()
            cur.execute(q)
            return pd.read_sql(q, conn)

        except (Exception, pg.DatabaseError) as error:
            print(error)

def run_command(c):
    with conn:
        try:
            cur = conn.cursor()
            cur.execute(c)
            cur.close()
            conn.commit()
            
        except (Exception, pg.DatabaseError) as error:
            print(error)
            
# reading in created datasets
track_metadata = pd.read_csv('../data/all_tracks_cleaned_V2.csv', index_col=0)
spotify_features = pd.read_csv('../data/audio_features_final.csv', index_col=0)

In [3]:
track_metadata.head()

,track_id,track_name,artist,artist_id,track_album_album_type,track_album_id,track_album_name,track_duration_ms,track_popularity,track_preview_url,subgenres,genre_1,genre_2,genre_3
0,6YDf6QV7QfCEo8O2dbWalK,Real Love Baby,Father John Misty,2kGBy2WHvF0VdZyqiVCkDT,single,6IpBMtAMrBvSHc9y6rTtQQ,Real Love Baby,189293.0,66.0,https://p.scdn.co/mp3-preview/a73451291399b823...,"['art pop', 'chamber pop', 'freak folk', 'indi...",art pop,chamber pop,freak folk
1,0qaWxVVd3KrP4WY27KWpIe,In a River,Rostam,04XggbrM51GcFPTxBYtRXT,single,4T3wWtIP6KqYa0KRZiuYiG,In a River,223573.0,58.0,https://p.scdn.co/mp3-preview/a47af0dc9a8a5582...,"['art pop', 'chamber pop', 'indie pop', 'indie...",art pop,chamber pop,indie pop
2,1yZMv2GMAibgLGsQiowZrt,Nancy From Now On,Father John Misty,2kGBy2WHvF0VdZyqiVCkDT,album,1BOfOlZo9Nzx7SmYAucY9t,Fear Fun,234160.0,59.0,https://p.scdn.co/mp3-preview/c209b9ccde129e96...,"['art pop', 'chamber pop', 'freak folk', 'indi...",art pop,chamber pop,freak folk
3,2eg2gvPXuwZ9FyrPaLgrXi,Chateau Lobby #4 (in C for Two Virgins),Father John Misty,2kGBy2WHvF0VdZyqiVCkDT,album,2A8IKX257C4hJaYUHMhLP7,"I Love You, Honeybear",170840.0,58.0,https://p.scdn.co/mp3-preview/1f9ea14fa26f55bb...,"['art pop', 'chamber pop', 'freak folk', 'indi...",art pop,chamber pop,freak folk
4,0wzruvvN7f5wu39aFcjTMw,Hollywood Forever Cemetery Sings,Father John Misty,2kGBy2WHvF0VdZyqiVCkDT,album,1BOfOlZo9Nzx7SmYAucY9t,Fear Fun,190613.0,56.0,https://p.scdn.co/mp3-preview/6aceffbff71010d0...,"['art pop', 'chamber pop', 'freak folk', 'indi...",art pop,chamber pop,freak folk


In [4]:
spotify_features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.644,0.912,9,-6.072,1,0.0384,0.00301,0.657000,0.0782,0.222,126.011,audio_features,0Tbgn7Ocm54WoqJUsIlfp4,spotify:track:0Tbgn7Ocm54WoqJUsIlfp4,https://api.spotify.com/v1/tracks/0Tbgn7Ocm54W...,https://api.spotify.com/v1/audio-analysis/0Tbg...,238668,4
1,0.696,0.623,7,-7.600,1,0.1260,0.01750,0.000091,0.1430,0.573,173.951,audio_features,1XibR7VrDllfBz6yuTndmb,spotify:track:1XibR7VrDllfBz6yuTndmb,https://api.spotify.com/v1/tracks/1XibR7VrDllf...,https://api.spotify.com/v1/audio-analysis/1Xib...,237241,4
2,0.700,0.722,7,-3.558,0,0.0369,0.22100,0.000000,0.2720,0.756,90.989,audio_features,3tjFYV6RSFtuktYl3ZtYcq,spotify:track:3tjFYV6RSFtuktYl3ZtYcq,https://api.spotify.com/v1/tracks/3tjFYV6RSFtu...,https://api.spotify.com/v1/audio-analysis/3tjF...,140526,4
3,0.779,0.705,0,-3.895,1,0.0344,0.36900,0.000008,0.1240,0.946,101.022,audio_features,27u7t9d7ZQoyjsCROHuZJ3,spotify:track:27u7t9d7ZQoyjsCROHuZJ3,https://api.spotify.com/v1/tracks/27u7t9d7ZQoy...,https://api.spotify.com/v1/audio-analysis/27u7...,178374,4
4,0.747,0.717,9,-4.841,0,0.1790,0.19900,0.000000,0.1320,0.523,150.965,audio_features,6piAUJJQFD8oHDUr0b7l7q,spotify:track:6piAUJJQFD8oHDUr0b7l7q,https://api.spotify.com/v1/tracks/6piAUJJQFD8o...,https://api.spotify.com/v1/audio-analysis/6piA...,179133,4


In [11]:
q = '''SELECT * FROM spotify_features LIMIT 5'''

res = run_query(q)
res

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature
0,0.417,0.687,7,-6.484,1,0.0372,0.0955,0.02780,0.0989,0.627,204.110,6YDf6QV7QfCEo8O2dbWalK,189293.0,4
1,0.493,0.437,7,-9.494,1,0.0326,0.3940,0.00197,0.2250,0.425,78.331,0qaWxVVd3KrP4WY27KWpIe,223573.0,4
2,0.534,0.659,0,-8.279,1,0.0270,0.4870,0.08080,0.0892,0.497,107.052,1yZMv2GMAibgLGsQiowZrt,234160.0,4
3,0.436,0.804,0,-4.907,1,0.0409,0.2090,0.00000,0.1710,0.784,188.061,2eg2gvPXuwZ9FyrPaLgrXi,170840.0,4
4,0.583,0.831,9,-3.417,1,0.0289,0.0502,0.27200,0.0400,0.628,94.009,0wzruvvN7f5wu39aFcjTMw,190613.0,4
